In [13]:
from qdrant_client import models, QdrantClient
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain.vectorstores.qdrant import Qdrant
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
#
from tqdm.auto import tqdm
from uuid import uuid4
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from sentence_transformers import SentenceTransformer, util
import pandas as pd

#
import os
import random

In [3]:
# Load the document that you need to parse, please change the location to where the pdf resides

# Load 1 PDF file
loader = PyPDFLoader("/mnt/data/smuckers_poc/RAG/2024-First-Quarter-Results.pdf")
# or load an entire folder
# loader = PyPDFDirectoryLoader("/mnt/data/RAG/")
data = loader.load_and_split(RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0))

In [5]:
print(f"There are {len(data)} pages in the document")

There are 37 pages in the document


In [10]:
# Pick a sample page
print(data[random.randint(0, len(data))])

page_content="increased \n$61.7 million\n, or 23 percent.\nInterest Expense and Income Taxes\nNet interest expense decreased \n$7.0 million\n, primarily due to an increase in interest income, reflecting higher interest rates as\ncompared to the prior year, and a decrease in interest expense related to the Company's commercial paper program, as there was\nno balance outstanding at the end of the quarter.\nThe effective income tax rate was 23.0 percent, compared to 22.2 percent in the prior year. The adjusted effective income tax rate\nwas 23.6 percent, compared to 23.0 percent in the prior year. The prior year effective income tax rates included a favorable\ndeferred tax benefit of a state income tax rate reduction.\nCash Flow and Debt\nCash provided by operating activities was \n$217.9 million\n, compared to cash used of \n$39.0 million\n in the prior year, primarily\nreflecting less cash required to fund working capital, lapping the \n$70.0 million\n contribution to the Company's U.S.

In [ ]:
metadatas = []
texts = []
for row in data:
  metadatas.append(row.metadata)
  texts.append(row.page_content)
print(len(metadatas),len(texts))

In [ ]:
prompt_template = """Use the following pieces of context to answer the question enclosed within  3 backticks at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Please provide an answer which is factually correct and based on the information retrieved from the vector store.
Please also mention any quotes supporting the answer if any present in the context supplied within two double quotes "" .

{context}

QUESTION:```{question}```
ANSWER:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context","question"])
#


In [ ]:
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

In [ ]:
embedding_model_name = "BAAI/bge-small-en"
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '/mnt/data/smuckers_poc/model_cache/'
embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en",
                                      model_kwargs=model_kwargs,
                                      encode_kwargs=encode_kwargs
                                     )

In [ ]:
doc_store = Qdrant.from_texts(texts,
                              metadatas=metadatas,
                              embedding=embeddings,
                              path="/mnt/data/smuckers_poc/local_qdrant/",
                              collection=f"{embedding_model_name}_press_release")

In [ ]:
chain_type_kwargs = {"prompt": PROMPT}


################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

model_id = "NousResearch/Llama-2-7b-chat-hf"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    cache_dir="/mnt/data/smuckers_poc/model_cache/",
    quantization_config=bnb_config,
    device_map=device_map
)

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
tokenizer.pad_token_id = model.config.eos_token_id

rag_llm = HuggingFacePipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    model_kwargs={
        "temperature": 0,
        "max_length": 100
    }
)
    
qa_chain = RetrievalQA.from_chain_type(llm=rag_llm,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=doc_store.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True
                                      )

In [ ]:
user_question = input("Please provide your question here :")
result = qa_chain(user_question)

In [ ]:
result['result']